In [11]:
import os
import requests
import pandas as pd

> SMART WATCH 

In [16]:
CATEGORY_ID = 8039
CATEGORY_NAME = "Smart Watches"
NUM_PAGES = 2
SAVE_DIR = "images/watch"
os.makedirs(SAVE_DIR, exist_ok=True)

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

records = []

# Function to download image
def download_image(url, save_path):
    try:
        response = requests.get(url, headers=HEADERS)
        if response.status_code == 200:
            with open(save_path, "wb") as f:
                f.write(response.content)
    except Exception as e:
        print(f"Error downloading image: {e}")

In [17]:
for page in range(1, NUM_PAGES + 1):
    api_url = f"https://tiki.vn/api/personalish/v1/blocks/listings?limit=40&category={CATEGORY_ID}&page={page}&sort_by=popularity"

    try:
        response = requests.get(api_url, headers=HEADERS)
        if response.status_code != 200:
            print(f"HTTP Error {response.status_code} on page {page}")
            continue

        data = response.json()
    except Exception as e:
        print("Unable to parse JSON:", e)
        print("Response:", response.text[:300])
        continue

    for item in data.get("data", []):
        product_id = item.get("id")
        name = item.get("name", "no_name")
        price = item.get("price", 0)
        image_url = item.get("thumbnail_url")

        if not image_url or not product_id:
            continue

        filename = f"{product_id}.jpg"
        full_path = os.path.join(SAVE_DIR, filename)
        download_image(image_url, full_path)

        records.append({
            "product_id": product_id,
            "category": CATEGORY_NAME,
            "product_name": name,
            "price": price,
            "image": filename,
            "source": image_url
        })

        print(f"Downloaded: {name}")


Downloaded: Dây Cáp Sạc Từ Tính cho Đồng Hồ Xiaomi Mi Band 5/ Mi Band 6/ Mi Band 7/ Mi band 8/ Mi band 9/ Mi band 10- Hàng chính hãng - Miband 5/6/7
Downloaded: Dây đeo cho vòng đeo tay thông minh Xiaomi MiBand 8/ Miband 9/ Mi band 10 ngàm thép_ Hàng Chính Hãng
Downloaded: Cáp sạc thay thế cho các dòng đồng hồ Garmin Fenix 7, Fenix 6, Fenix 5, Forerunner 955, 945, 935, 245, 265, 255, 45, 645, Instinct, Viviactive 3, Venu/ Garmin Epix Gen 2- Hàng Nhập Khẩu 
Downloaded: Vòng Đeo Tay Thông Minh Xiaomi Smart Band 9 Active – Hàng Chính Hãng
Downloaded: Dock Sạc Cho Huawei Watch GT 2 Pro / GT Runner / GT3/3Pro / GT4/4Pro / Huawei Watch GT5 / Huawei Watch GT 5 Pro - Hàng Chính Hãng
Downloaded: Đồng Hồ Thông Minh Xiaomi Redmi Watch 5 ACTIVE – Hàng Chính Hãng
Downloaded: Cáp Sạc Đế Sạc cho Samsung Galaxy Watch 3/4/4Classic/5/5Pro/6/6Classic/7/Ultra & Galaxy Watch Active 1/2- Hàng Chính Hãng
Downloaded: Apple Watch Series 10 GPS (Viền nhôm, dây cao su)
Downloaded: Apple Watch SE 2024 GPS (Viền n

In [18]:
df = pd.DataFrame(records)
df.to_excel("data.xlsx", index=False)